# A joint model CAR (model 2) 
## Common Markov Gaussian Random Field
 $$ [P , S ] = [P | GMRF] [ S | GMRF] $$

In [13]:
## Import code:
setwd('/apps/external_plugins/biospytial_rwrapper/biospytial.rwrapper/R/')

In [14]:
rm(list=ls())                                                                                     
source("init_data.R")                                                                             
# load the building function                                                                      
source("joint.binomial.bymCAR.R")                                                                 
n.sample = 10000                                                                                  
burnin=10000                                                                                      
postburnin = burnin +1000                                                                         
thin = 1                                                                                          
verbose = TRUE                                                                                    
results  <- joint.binomial.bymCARModel2(formula_S = formula_sample, 
                                        formula_P = formula_presence,
                                        n.sample=n.sample,
                                        burnin=burnin,
                                        postburnin=postburnin,
                                        thin=thin,
                                        verbose=TRUE) 


Setting up the model.
Performing burnin period for Sample Effort Model 
  |======================================================================| 100%
Summarising results.
Finished in  124.3 seconds.
Setting up the model.
Performing burnin period for Presence model 
  |======================================================================| 100%
Summarising results.
Finished in  124 seconds.
Sampling from the joint model 10000 post burnin and thinned (if requested) samples.
  |======================================================================| 100%
[1] "Compiling summary for S process"
[1] "Compiling summary for the P process"


## Remember to check likelihood modelfit

In [22]:
results$S$modelfit

DIC           p.d          WAIC           p.w          LMPL 
          Inf           Inf   100023566.7    50004973.2    -1895639.8 
loglikelihood 
    -546293.8

In [23]:
results$P$modelfit

DIC           p.d          WAIC           p.w          LMPL 
    49131.541      2245.328   3281843.586   1640390.092   -113899.684 
loglikelihood 
   -22320.443

In [26]:
results$S$summary.results

,Median,2.5%,97.5%,n.sample,% accept,n.effective,Geweke.diag
(Intercept),1.113870e+01,11.1387,1.113870e+01,10000,54.3,380.0,-1.0
Disttoroadm,-8.000000e-04,-0.0008,-8.000000e-04,10000,54.3,434.0,0.8
Populationm,-1.000000e-04,-0.0001,-1.000000e-04,10000,54.3,1096.1,1.1
tau2,6.008482e+09,206.2353,6.758887e+10,10000,100.0,1.8,-2.1
sigma2,1.675376e+10,2.2658,1.871404e+11,10000,100.0,1.8,-2.2


In [ ]:
post.sample_p = data.frame(results$S$samples$fitted)
post.presence_p = data.frame(results$P$samples$fitted)

In [ ]:
pss  <- post.sample_p %>% mutate_all(function(p) rbernoulli(1,p))
pps <- post.presence_p %>% mutate_all(function(p) rbernoulli(1,p))                                        

In [5]:

sumpps <- colSums(pps)

In [6]:
sumpss <- colSums(pss)

In [7]:
prob_pps <- sumpps / length(pps)

In [8]:
prob_pss <- sumpss / length(pss)

In [9]:
probpres = prob_pps * prob_pss

In [10]:
## save to posterity
# Save output sample to display in python
## Save the complete chain trace for analysing in Python,
TDF = read.csv("/outputs/training_data_sample_puebla_p9_abies_pinophyta.csv")
## Order it according to the id of the cell
## This is important because the adjancy matrix rows need to be the same
TDF = TDF[order(TDF$cell_ids),]

In [11]:
chain = cbind(TDF$cell_ids,probpres,prob_pps,prob_pss)
file_ = '/outputs/presence_only_models/modelCAR_model2_fitted.csv'
write.csv(chain,file_)